# Métodos iterativos

In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

#### Observaciones


* $\texttt{np.tril}$ devuelve una copia de la matriz $A$, pero con los que estan por encima de la diagonal de la matriz hechos $0$.
* $\texttt{np.triu}$ devuelve una copia de la matriz $A$, pero con los que estan por debajo de la diagonal de la matriz hechos $0$.
* $\texttt{np.diag}$:
  - Si el input es una matriz $A$, devuelve su diagonal como un vector $v$ de una dimensión.
  - Si el input es un vector $v$ de una dimensión, construye una matriz diagonal $A$ con $v$ como diagonal.
 
Clarifico esto con un par de ejemplos:   

In [2]:
# Defino una matriz para ejemplificar
A = np.asarray([[1,2,3],[4,5,6],[7,8,9]])

In [3]:
# Me quedo con la parte inferior + diagonal de la matriz
np.tril(A)

array([[1, 0, 0],
       [4, 5, 0],
       [7, 8, 9]])

In [4]:
# Me quedo con la parte inferior de la matriz
np.tril(A,-1)

array([[0, 0, 0],
       [4, 0, 0],
       [7, 8, 0]])

In [5]:
# Me quedo con la parte superior + diagonal de la matriz
np.triu(A)

array([[1, 2, 3],
       [0, 5, 6],
       [0, 0, 9]])

In [6]:
# Me quedo con la parte superior de la matriz
np.triu(A,1)

array([[0, 2, 3],
       [0, 0, 6],
       [0, 0, 0]])

In [7]:
# Me quedo con la diagonal de la matriz
np.diag(A)

array([1, 5, 9])

In [8]:
# Me quedo con la diagonal de la matriz como matriz
np.diag(np.diag(A))

array([[1, 0, 0],
       [0, 5, 0],
       [0, 0, 9]])

In [9]:
# Me quedo con la inversa de la diagonal como matriz
np.diag(1/np.diag(A))

array([[1.        , 0.        , 0.        ],
       [0.        , 0.2       , 0.        ],
       [0.        , 0.        , 0.11111111]])

## Método de Jacobi

En el método de Jacobi, escribimos a la matriz $A$ como

$$A=D+R$$

donde $D$ es diagonal y $R$ es la suma de una matriz triangular inferior $L$ y una matriz triangular superior $U$, es decir $R=L+U$. Para resolver $Ax=b$, tenemos:

$$(D+R)x=b$$
$$Dx + Rx = b$$

y por lo tanto,

$$x = D^{-1} (b-Rx)$$

Si fuera que $a_{ii}=0$ para todo $i$, entonces la regla iterativa del método de Jacobi puede ser definida como:

$$x^{(k+1)} = D^{-1} \left(b-Rx^{(k)}\right)$$

o de otra manera:

$$x_i^{(k+1)} = \frac{1}{a_{ii}} \left(b_i - \sum_{j\neq i} a_{ij}x_j^{(k)} \right) \, \text{, para todo } i$$

In [10]:
def Jacobi(A, b, tol = 1e-10, tope_iteraciones = 1000):
    # Chequeamos que las dimensiones sean apropiadas
    if A.shape[0] != A.shape[1]:
        raise ValueError("La matriz A no es cuadrada")
    if A.shape[1] != b.shape[0]:
        raise ValueError("Las dimensiones del sistema no cuadran")

    # Definimos a n como la dimensión de la matriz
    n = A.shape[0]

    # Inicializamos el vector respuesta
    x = np.zeros(n)

    # Definimos matrices de interés
    D_inv = np.diag(1 / np.diag(A))
    R = np.tril(A,-1) + np.triu(A,1)

    # Definimos el tope de iteraciones
    tope_iteraciones = 1000

    k = 0
    while k < tope_iteraciones:
        x_nuevo = D_inv @ (b - R@x)

        # Si el método se estacionó
        if np.linalg.norm(x_nuevo - x, ord = np.inf) < tol:
            return x_nuevo

        # Actualizamos el valor de x
        x = x_nuevo
        k += 1
        
    raise RuntimeError(f"Advertencia: se alcanzó el tope de {tope_iteraciones} iteraciones sin converger.")

#### Ejemplo 1

Tomemos la matriz estrictamente diagonal dominante:

$$
A
=
\begin{pmatrix}
10 & 2 & 1\\
1 & 10 & 2\\
2 & 3 & 10\\
\end{pmatrix}
$$

que por lo probado teóricamente debería converger.

In [11]:
# Defino una matriz y un vector para probar la función

A = np.asarray([[10,2,1],[1,10,2],[2,3,10]])
b = np.array([7,8,9])

In [12]:
# Vemos la solución que nos da Python con numpy
np.linalg.solve(A,b)

array([0.51372119, 0.62678375, 0.60922064])

In [13]:
# Vemos la solución que nos da nuestro método de Jacobi
Jacobi(A,b)

array([0.51372119, 0.62678375, 0.60922064])

Vemos, que como teníamos probado teóricamente, el método parece converger a una buena solución

### Ejemplo 2

Tomemos la matriz
$$
A
=
\begin{pmatrix}
0.3 & 0.1 & 0.2 \\
0.1 & 0.4 & 0.1 \\
0.2 & 0.1 & 0.3
\end{pmatrix}
$$
que, como veremos, cumple $\rho(A) < 1$ y por tanto debería converger

In [14]:
# Defino una matriz y un vector para probar la función

A = np.asarray([[0.3,0.1,0.2],[0.1,0.4,0.1],[0.2,0.1,0.3]])
b = np.array([7,8,9])

In [15]:
# Aseveramos que el radio espectral de A es menor que 1
assert np.max(np.abs(np.linalg.eigvals(A))) < 1

In [16]:
# Vemos la solución que nos da Python con numpy
np.linalg.solve(A,b)

array([ 3.33333333, 13.33333333, 23.33333333])

In [17]:
# Vemos la solución que nos da nuestro método de Jacobi
Jacobi(A,b)

array([ 3.33333333, 13.33333333, 23.33333333])

Observamos, entonces, el comportamiento esperado.

#### Ejemplo 3

Si tomamos la matriz
$$
A
=
\begin{pmatrix}
1 & 2 & 3 \\
4 & 5 & 6 \\
7 & 8 & 10 \\
\end{pmatrix}
$$

observaremos que al no cumplir ninguna de las condiciones de Jacobi, no convergerá a una buena solución.

In [18]:
# Defino otra matriz para probar cuando Jacobi no converge

A = np.asarray([[1,2,3],[4,5,6],[7,8,10]])
b = np.array([7,8,9])

In [19]:
# Vemos la solución que nos da nuestro método de Jacobi
Jacobi(A,b)

C:\Users\Fausto\AppData\Local\Temp\ipykernel_13044\2618127736.py:23: RuntimeWarning: overflow encountered in matmul
  x_nuevo = D_inv @ (b - R@x)
C:\Users\Fausto\AppData\Local\Temp\ipykernel_13044\2618127736.py:23: RuntimeWarning: invalid value encountered in matmul
  x_nuevo = D_inv @ (b - R@x)


RuntimeError: Advertencia: se alcanzó el tope de 1000 iteraciones sin converger.

Obtenemos la advertencia de que llegamos al tope de iteraciones que habíamos planteado para la función, pero, ¿y si forzamos más iteraciones?

In [20]:
# Vemos la solución que nos da nuestro método de Jacobi forzando más iteraciones
Jacobi(A,b,10000)

array([7. , 1.6, 0.9])

In [21]:
np.linalg.solve(A,b)

array([-6.33333333e+00,  6.66666667e+00,  1.77635684e-15])

Vemos que las soluciones en este caso son significativamente distintas. Esto nos confirma que para la utilización de métodos iterativos deberemos ser cautelosos sobre las condiciones que hay que imponerle a la matriz para que converjan.

## Método de Gauss-Seidel